In [1]:
import ast
import glob
import gzip
import json
import os
import sys
from multiprocessing import Pool

import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

import process
import testing
from haystack import Pipeline
from haystack.document_store import FAISSDocumentStore
from haystack.reader import FARMReader
from haystack.retriever.dense import DensePassageRetriever
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader



/home/npateel/anaconda3/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


ImportError: cannot import name 'clean_wiki_text' from 'haystack.utils' (/home/npateel/anaconda3/lib/python3.8/site-packages/haystack/utils.py)

In [2]:
year = "2008"
dataset = "nq_smaller.json"
output_path = "hi.json"
faiss = os.path.join(year, 'wiki_dump_embeddings.faiss')
sql_url = 'sqlite:///' + os.path.join(year, 'faiss_document_store.db')

document_store = FAISSDocumentStore.load(faiss,
                                         sql_url=sql_url,
                                         index='document')
print("Loaded document store")
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    batch_size=64,
    embed_title=True)

print("Loaded retriever")
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2",
                    use_gpu=True,
                    no_ans_boost=-10,
                    context_window_size=500,
                    top_k_per_candidate=3,
                    top_k_per_sample=1,
                    num_processes=1,
                    max_seq_len=256,
                    doc_stride=128,
                    progress_bar=False)
print("Loaded reader")

p = ExtractiveQAPipeline(reader, retriever)
# p.add_node(component=retriever, name="ESRetriever1", inputs=["Query"])
# p.add_node(component=reader, name="QAReader", inputs=["ESRetriever1"])


NameError: name 'os' is not defined

In [24]:
dir(retriever)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_dispatch_run',
 '_get_predictions',
 'batch_size',
 'devices',
 'document_store',
 'embed_passages',
 'embed_queries',
 'eval',
 'get_subclass',
 'index_count',
 'index_time',
 'infer_tokenizer_classes',
 'load',
 'load_from_args',
 'load_from_pipeline_config',
 'model',
 'name',
 'outgoing_edges',
 'passage_encoder',
 'passage_tokenizer',
 'pipeline_config',
 'print_time',
 'processor',
 'progress_bar',
 'query_count',
 'query_encoder',
 'query_time',
 'query_tokenizer',
 'retrieve',
 'retrieve_time',
 'run',
 'run_indexing',
 'run_query',
 'save',
 'set_config',
 'subclasses',
 'timing',
 'top_k',


In [25]:
x = retriever.document_store.get_all_documents_generator()
print(next(x))

{'text': "Petersville Australia Limited was an Australian public company which specialized in food product manufacture, distribution and marketing until its takeover by Adelaide Steamship Company in the 1980s.\nPetersville was noted for establishing Peters Ice Cream in the 1930s by its founder Fred Peters. It diversified into all areas of dairy food manufacture by the 1960s. It merged with H.C. Sleigh Co. in the 1970s. After its takeover by Adelaide Steamship, Petersville's divisions were broken up, notably the ice cream division was sold to the Swiss international food giant Nestlé.\nSome of the foods it produced or distributed were: ice cream, yoghurt, cheese, butter and imported cheeses. Its head office (now Nestle Australia's) was in Wellington Road, Mulgrave, Victoria, Australia.\nPetersville had Dairy production plants at Warragul, Trafalgar and Yarragon in Victoria and at Orange, Taree and Grafton in New South Wales.", 'score': None, 'question': None, 'meta': {'vector_id': '0', 

In [44]:
next(x)

{'text': 'Judy Bernly was a fictional character in the movie Nine to Five as well as the short-lived television series of the same name. In the movie, she was played by Jane Fonda and in the television series, she was played by Valerie Curtin. Ironically, Jane Fonda also helped produce the early seasons of the TV series, and played a security guard, named O\'Neill, as well. \nCharacter.\nA college graduate, Judy Bernly was also a recent divorcee, her husband, Dick Radman (Lawrence Pressman), had divorced her after he had an affair with his secretary, Liza; and having just moved to a new apartment, desperately needed a job. She joined Consolidated Companies working in the same section as Violet Newstead (Lily Tomlin) who became one of her best friends. She also made friends with several of her office mates, namely a nice woman in her section named Maria Del Gado, who helped her get used to the harshness of office life.\nAt first, she wasn\'t used to being around Doralee Rhodes (Dolly Pa